In [2]:
import os 
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.engine.url import URL
import numpy as np
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import boto3
from scipy.stats import pearsonr
from scipy.stats import gmean
import statsmodels.api as sm

In [3]:
import sys
sys.path.append("/Users/sebastianbedoyamazo/Documents/siwa_git/iluma-kb")
import utils_schema as us
from utils_schema import get_otu_and_taxo
from utils_schema import get_mapfile

In [4]:
us

<module 'utils_schema' from '/Users/sebastianbedoyamazo/Documents/siwa_git/iluma-kb/utils_schema.py'>

In [5]:
load_dotenv("/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others/.env", override=True)

True

In [6]:
os.environ
sys.path.append("/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others")
import utils_dataexp as u
u

Reading aws keys from .env successfully!!


<module 'utils_dataexp' from '/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others/utils_dataexp.py'>

In [7]:
s3_client = boto3.client("s3")
s3_resource = boto3.resource("s3")

In [8]:
url = URL.create(
    drivername='redshift+redshift_connector',
    host='iluma-kb-1.cn4ff1ztoyt9.us-east-1.redshift.amazonaws.com',
    port=5439,
    database='dev',
    username='awsuser',
    password='Ilumasiwa1'
)

engine = create_engine(url)
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

redshift_metadata = sa.MetaData(bind=session.bind)

In [9]:
#con fieldale
metadata_all = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Joyce and Fieldale/Fieldale/meta_exp_broiler.csv")
metadata_all

,SampleID,SampleLocation,ProjectID,Age,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR,Treatment,cat_1_BW,cat_1_BWGbefore,cat_1_BWG,cat_1_FCRbefore,cat_1_FCR,House
0,0008_01C,cecum,E321,14,1,2,1,390.3333,NaN,NaN,NaN,NaN,2,Medium,NaN,NaN,NaN,NaN,NaN
1,0008_01F,feces,E321,14,1,2,1,390.3333,NaN,NaN,NaN,NaN,2,Medium,NaN,NaN,NaN,NaN,NaN
2,0008_02C,cecum,E321,14,2,2,3,383.8333,NaN,NaN,NaN,NaN,2,Low,NaN,NaN,NaN,NaN,NaN
3,0008_02F,feces,E321,14,2,2,3,383.8333,NaN,NaN,NaN,NaN,2,Low,NaN,NaN,NaN,NaN,NaN
4,0008_02I,ileum,E321,14,2,2,3,383.8333,NaN,NaN,NaN,NaN,2,Low,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,0226_12C,C,Fieldale,28,23,B,12,NaN,NaN,NaN,1.466042,NaN,Fractal,NaN,NaN,NaN,Medium,NaN,1.0
2003,0225_01I,I,Fieldale,28,1,A,1,NaN,NaN,NaN,1.546512,NaN,Control,NaN,NaN,NaN,Medium,NaN,1.0
2004,0225_02I,I,Fieldale,28,3,A,2,NaN,NaN,NaN,1.483436,NaN,Control,NaN,NaN,NaN,Medium,NaN,1.0
2005,0225_03I,I,Fieldale,28,5,A,3,NaN,NaN,NaN,1.521951,NaN,Control,NaN,NaN,NaN,Medium,NaN,1.0


In [11]:
samples = metadata_all.SampleID

In [12]:
query = "SELECT * FROM microbiome;"
metadata = pd.read_sql(query, engine)
print(metadata.shape)
print(metadata.columns)
metadata.head()
#metadata.groupby(['treatment', 'treatmentnumber']).count()

(4765, 8)
Index(['sampleid', 'fullsampleid', 'kitid', 'runid', 'animalid',
       'samplelocation', 'alphashannon', 'alphaobserved'],
      dtype='object')


,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved
0,H001_01F,H001_01F-M,H001,20181130,H001_01,feces,2.429103,27.0
1,H001_01I,H001_01I-M,H001,20181130,H001_01,ileum,1.339185,47.0
2,H001_04C,H001_04C-M,H001,20181130,H001_04,cecum,6.082133,125.0
3,H001_04F,H001_04F-M,H001,20181130,H001_04,feces,2.705841,20.0
4,H001_05F,H001_05F-M,H001,20181130,H001_05,feces,0.940546,9.0


In [13]:
metadata = metadata[['sampleid', 'alphashannon',
       'alphaobserved']]
metadata.head()

,sampleid,alphashannon,alphaobserved
0,H001_01F,2.429103,27.0
1,H001_01I,1.339185,47.0
2,H001_04C,6.082133,125.0
3,H001_04F,2.705841,20.0
4,H001_05F,0.940546,9.0


In [14]:
metadata = metadata.rename(columns={'sampleid': 'SampleID'})
metadata

,SampleID,alphashannon,alphaobserved
0,H001_01F,2.429103,27.0
1,H001_01I,1.339185,47.0
2,H001_04C,6.082133,125.0
3,H001_04F,2.705841,20.0
4,H001_05F,0.940546,9.0
...,...,...,...
4760,0170_07I,3.200000,66.0
4761,0170_08C,5.690000,200.0
4762,0170_08F,4.870000,88.0
4763,0170_09F,2.440000,62.0


In [15]:
metadata_alpha = pd.merge(metadata_all, metadata, on =['SampleID'], how='left')
print(metadata_alpha.shape)
print(metadata_alpha.columns)
metadata_alpha.head(3)

(2007, 21)
Index(['SampleID', 'SampleLocation', 'ProjectID', 'Age', 'Pen', 'Trt', 'Block',
       'BW', 'BWGbefore', 'BWG', 'FCRbefore', 'FCR', 'Treatment', 'cat_1_BW',
       'cat_1_BWGbefore', 'cat_1_BWG', 'cat_1_FCRbefore', 'cat_1_FCR', 'House',
       'alphashannon', 'alphaobserved'],
      dtype='object')


,SampleID,SampleLocation,ProjectID,Age,Pen,Trt,Block,BW,BWGbefore,BWG,...,FCR,Treatment,cat_1_BW,cat_1_BWGbefore,cat_1_BWG,cat_1_FCRbefore,cat_1_FCR,House,alphashannon,alphaobserved
0,0008_01C,cecum,E321,14,1,2,1,390.3333,NaN,NaN,...,NaN,2,Medium,NaN,NaN,NaN,NaN,NaN,6.477676,214.0
1,0008_01F,feces,E321,14,1,2,1,390.3333,NaN,NaN,...,NaN,2,Medium,NaN,NaN,NaN,NaN,NaN,3.555358,53.0
2,0008_02C,cecum,E321,14,2,2,3,383.8333,NaN,NaN,...,NaN,2,Low,NaN,NaN,NaN,NaN,NaN,6.360295,212.0


In [16]:
metadata_alpha.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/correlation/alpha/metadata_alpha.csv", header=True, index=False)